<a href="https://colab.research.google.com/github/1024andrew/colab_ML-DL_practice/blob/main/8_%ED%8A%B8%EB%A6%AC%EC%9D%98_%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
wine = pd.read_csv("https://bit.ly/wine_csv_data")

In [ ]:
data = wine[["alcohol","sugar", "pH"]].to_numpy()
target = wine["class"].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import cross_validate

#랜덤 포레스트 불러오기
from sklearn.ensemble import RandomForestClassifier
import numpy as np

rf = RandomForestClassifier(n_jobs=-1,random_state=42)

#return_train_score을 True로 설정함으로써 훈련세트에 대한 점수를 확인할 수 있다.
#test_score는 검증 세트로 검증한 점수임을 잊지말아야 한다.
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores["train_score"]))
print(np.mean(scores["test_score"]))

0.9973541965122431
0.8905151032797809


In [ ]:
#cross_validate를 통해서 교차검증을 끝냈을 때는 설정된 하이퍼 파라미터의 성능이 이정도일 것이라는 추정치를 보여주는 것일 뿐이다
#GridSearchCV는 교차 검증과 달리 내가 지정한 파라미터 범위 내에서 최적의 하이퍼 파라미터 설정을 찾아준다.

#따라서 fit 으로 모델을 훈련하는 과정이 반드시 필요하다.
rf.fit(train_input, train_target)


#랜덤 포레스트는 결정트리가 제공하는 중요한 매개변수를 모두 제공한다.
#feature_importances_ 로 특성 중요도(처음에 data로 분류한 종목들)를 확인할 수 있다.
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [ ]:
#랜덤 포레스트에는 OOB 샘플이라는 것이 존재한다. OOB란 부트스트랩 샘플에 속하지 않는 샘플들을 모아놓은 것이다.
#랜덤 포레스트는 OOB 샘플을 이용하여 자체적으로 모델을 평가할 수 있다.
#OOB가 존재하지 않을 수도 있는데 oob_score을 계산할 때 그런 샘플은 평가에서 제외한다.

#OOB로 모델을 평가하려면 oob_score을 True 로 지정해 주어야 한다.
#사용할 때는 oob_score_ 로 사용한다.
rf = RandomForestClassifier(n_jobs= -1, oob_score=True, random_state=42)

#랜덤 포레스트를 훈련 세트를 통해 훈련시키고(이 과정에서 부트스트랩 샘플 생성 및 사용) 자체 OOB 샘플을 사용해 평가하는 과정
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


In [ ]:
#엑스트라 트리
#엑스트라 트리는 랜덤 포레스트와 유사하지만 부트스트랩 샘플을 사용하지 않는다는 차이점이 있다.
#엑스트라 트리는 훈련 세트를 사용해 결정 트리를 만들고 특성을 무작위로 선택하여 분할한다.
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)

#엑스트라 트리를 교차 검증하고 테스트 평균 점수와 검증 평균 점수를 출력
scores = cross_validate(et, train_input, train_target, n_jobs=-1, return_train_score=True)
print(np.mean(scores["train_score"]))
print(np.mean(scores["test_score"]))

0.9974503966084433
0.8887848893166506


In [ ]:
et.fit(train_input, train_target)

#엑스트라 역시 특성 중요도를 출력할 수 있다.
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


In [ ]:
#그레이디언트 부스팅
#길이가 얕은 결정 트리를 사용하여 이전의 오차를 보완하는 방식으로 앙상블 하는 방법이다.
#결정트리를 계속 추가하면서 경사 하강법을 사용한다.

from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, n_jobs=-1, return_train_score=True)

print(np.mean(scores["train_score"]))
print(np.mean(scores["test_score"]))

0.8881086892152563
0.8720430147331015


In [ ]:
#그레이디언트 부스팅은 결정 트리의 개수를 늘려도 과대적합을 잘 억제한다.
#학습률은 각 트리의 기여도를 의미한다.(커질수록 성큼성큼 가고 작을수록 살짝살짝씩 가는 느낌)

#결정트리의 개수는 n_estimators 로, 학습률은 learning_rate로 정한다.
gb = GradientBoostingClassifier(random_state=42, n_estimators=500, learning_rate=0.2)
scores=cross_validate(gb, train_input, train_target, n_jobs=-1, return_train_score=True)

print(np.mean(scores["train_score"]))
print(np.mean(scores["test_score"]))

0.9464595437171814
0.8780082549788999


In [ ]:
#마찬가지로 특성 중요도를 추출할 수 있다.

gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15887763 0.6799705  0.16115187]


In [ ]:
#히스토그램 기반 그레이디언트 부스팅
#히스토그램 기반 그레이디언트 부스팅에서 결정 트리 개수를 지정할 때는 그레이디언트 부스팅과는 다르게 max_iter 매개변수를 사용한다.
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, n_jobs=-1, return_train_score=True)

print(np.mean(scores["train_score"]))
print(np.mean(scores["test_score"]))

0.9321723946453317
0.8801241948619236


In [ ]:
#역시 특성 중요도를 출력할 수 있지만 permutation_importance() 라는 함수를 사용해야한다.
#permutation_importance()는 함수의 특성을 랜덤하게 섞어 모델의 성능 변화를 관찰하여 특성 중요도를 계산한다.
#n_repeats로 랜덤하게 섞는 횟수를 정할 수 있다(기본값 5)

from sklearn.inspection import permutation_importance

#permutation_importance 함수는 이미 학습된 모델의 특성 중요도를 평가하는 함수이므로 반드시 모델을 fit 한 상태에서 사용해야한다.
hgb.fit(train_input, train_target)

results = permutation_importance(hgb, train_input, train_target, random_state=42, n_jobs=-1, n_repeats=10)

#permutation_importance 함수가 반환하는 객체는 특성 중요도(importances) -> 원본 데이터 / 특성 중요도 평균(importances_mean) -> 주로 사용 / 특성 중요도 표준편차(importances_std) 이다.
print(results.importances_mean)

[0.08876275 0.23438522 0.08027708]
